In [19]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pickle
import random
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from nltk.classify import ClassifierI   #to inherit from nltk classifier class
from statistics import mode

In [2]:
path ='../Datasets/rawData/'
training_data = pd.read_csv(path + 'train.csv'.format(1))
training_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
label_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
label_names

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [4]:
text_only = training_data['comment_text']
text_only.head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

In [10]:
#nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

Hit Enter to continue: all
  [ ] conll2007........... Dependency Treebanks from CoNLL 2007 (Catalan
                           and Basque Subset)
  [ ] crubadan............ Crubadan Corpus
  [ ] dependency_treebank. Dependency Parsed Treebank
  [ ] dolch............... Dolch Word List
  [ ] europarl_raw........ Sample European Parliament Proceedings Parallel
                           Corpus
  [ ] floresta............ Portuguese Treebank
  [ ] framenet_v15........ FrameNet 1.5
  [ ] framenet_v17........ FrameNet 1.7
  [ ] gazetteers.......... Gazeteer Lists
  [ ] genesis............. Genesis Corpus
  [ ] gutenberg........... Project Gutenberg Selections
  [ ] ieer................ NIST IE-ER DATA SAMPLE
  [ ] inaugural........... C-Span Inaugural Address Corpus
  [ ] indian.............. Indian Language POS-Tagged Corpus
  [ ] jeita............... JEITA Public Morphologically Tagged Corpus (in
                           ChaSen format)
  [ ] kimmo............... PC-KIMMO Data Files
  [ ]

       |   Unzipping corpora/crubadan.zip.
       | Downloading package dependency_treebank to
       |     /home/user/nltk_data...
       |   Unzipping corpora/dependency_treebank.zip.
       | Downloading package dolch to /home/user/nltk_data...
       |   Unzipping corpora/dolch.zip.
       | Downloading package europarl_raw to /home/user/nltk_data...
       |   Unzipping corpora/europarl_raw.zip.
       | Downloading package floresta to /home/user/nltk_data...
       |   Unzipping corpora/floresta.zip.
       | Downloading package framenet_v15 to /home/user/nltk_data...
       |   Unzipping corpora/framenet_v15.zip.
       | Downloading package framenet_v17 to /home/user/nltk_data...
       |   Unzipping corpora/framenet_v17.zip.
       | Downloading package gazetteers to /home/user/nltk_data...
       |   Unzipping corpora/gazetteers.zip.
       | Downloading package genesis to /home/user/nltk_data...
       |   Unzipping corpora/genesis.zip.
       | Downloading package gutenberg

       | Downloading package bllip_wsj_no_aux to
       |     /home/user/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package word2vec_sample to
       |     /home/user/nltk_data...
       |   Unzipping models/word2vec_sample.zip.
       | Downloading package panlex_swadesh to /home/user/nltk_data...
       | Downloading package mte_teip5 to /home/user/nltk_data...
       |   Unzipping corpora/mte_teip5.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /home/user/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package perluniprops to /home/user/nltk_data...
       |   Unzipping misc/perluniprops.zip.
       | Downloading package nonbreaking_prefixes to
       |     /home/user/nltk_data...
       |   Unzipping corpora/nonbreaking_prefixes.zip.
       | Downloading package vader_lexicon to /home/user/nltk_data...
       | Downloading package porter_test to /home/user/nltk_data

True

In [5]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{'those', 'same', 'at', 'other', 'theirs', 'being', 'ours', 'a', "she's", 'why', 'having', 'after', 'its', 'until', 'over', "hasn't", 'weren', 'yourselves', 'only', "weren't", 'whom', 'mustn', 'here', "wouldn't", 'there', "you're", 'from', 'she', 'ain', 'd', 'who', 'just', "wasn't", 'won', 'wouldn', 'aren', 'that', 'it', 'your', 'be', 'and', 'to', 'yours', "mightn't", 'no', 'has', 'below', 'which', "that'll", 'once', 'will', 'into', 'their', "isn't", 'had', 'before', 'hers', 'herself', "don't", 'been', 'above', 'y', 'are', 'few', 'me', 'is', 'hasn', 'was', 'itself', 'out', 'some', "mustn't", 'hadn', 'further', 'do', 'than', 'shouldn', 'ma', "needn't", 'but', "should've", 'him', 'mightn', 'under', "you'd", 'they', 'haven', 'were', 'doing', 'themselves', 'own', 'am', 'did', 'because', 'the', 'should', 'can', 'we', 'his', 'wasn', 'i', 'most', 'have', 'if', 'too', 'each', 'her', 'any', 'such', 't', 'up', 'then', 'during', 'down', 'how', 'by', 'couldn', 'where', 'you', 'for', 'on', 'both', 

In [6]:
text_only[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [6]:
# def tokenizeWords(example_sentence):
#     words = word_tokenize(example_sentence)
#     filtered_sentence = []
#     for w in words:
#         if w not in stop_words:
#             filtered_sentence.append(w)
#     return filtered_sentence

# filtered_text=[]
# for i in range(len(text_only)):
#     filtered_text.append(tokenizeWords(text_only[i]))
# print(filtered_text[0:5])

In [49]:
all_words=[]
def tokenizeWords(example_sentence):
    words = word_tokenize(example_sentence)
    filtered_sentence = []
    for w in words:
        if w not in stop_words:
            all_words.append(w)

for i in range(len(text_only[:1])):
    tokenizeWords(text_only[i])

print(all_words)

['Explanation', 'Why', 'edits', 'made', 'username', 'Hardcore', 'Metallica', 'Fan', 'reverted', '?', 'They', "n't", 'vandalisms', ',', 'closure', 'GAs', 'I', 'voted', 'New', 'York', 'Dolls', 'FAC', '.', 'And', 'please', "n't", 'remove', 'template', 'talk', 'page', 'since', 'I', "'m", 'retired', 'now.89.205.38.27']


In [10]:
#all_words[0:5]
toxic_data = training_data['toxic']
print(len(toxic_data))

159571


In [11]:
documents = []
for i in range(len(toxic_data)):
    documents.append((text_only[i],toxic_data[i]))

save_documents = open("documents.pickle","wb")
pickle.dump(documents,save_documents)
save_documents.close()

print(len(documents))

159571


In [44]:
documents[0]

("Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
 0)

In [47]:
training_data['comment_text'].iloc[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [12]:
all_words = nltk.FreqDist(all_words)
len(all_words)

50223

In [13]:
word_features = list(all_words.keys())[:100]  #we don't need the count here for classification, only the words

save_word_features = open("wordFeatures.pickle","wb")
pickle.dump(word_features,save_word_features)
save_word_features.close()

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]
print(len(featuresets))

159571


In [16]:
random.shuffle(featuresets)

In [36]:
featuresets[0:5]

[({"'fact": False,
   "'fantasy": False,
   '//findarticles.com/p/articles/mi_hb039/is_/ai_n28889291': False,
   '159': False,
   '60,000': False,
   'ARMS': False,
   'Adriatic': False,
   'Angika': False,
   'Apollonian': False,
   'Besant': False,
   'Bug3r3r': False,
   'Cant': False,
   'Career': False,
   'Conner': False,
   'Contributions/': False,
   'Cooperstown': False,
   'Count': False,
   'Delorko': False,
   'Edd': False,
   'Eddie': False,
   'Fürst': False,
   'Gateman': False,
   'Go': False,
   'Gordons': False,
   'Graziano': False,
   'Guesipe': False,
   'Inflammatory': False,
   'Kaplan': False,
   'Karakalpaks': False,
   'LVATT': False,
   'MaM': False,
   'Merv': False,
   'Moonlite': False,
   'Msa': False,
   'Perestroika': False,
   'Poop': False,
   'Predatory': False,
   'Racial': False,
   'Scriptures': False,
   'Spatharius': False,
   'Summon': False,
   'Sutlej': False,
   'TRIVIA': False,
   'UVE': False,
   'Venus': False,
   'Well': False,
   'Winds

## Naive Baye's

In [17]:
training_set = featuresets[:10000]
testing_set = featuresets[10000:20000]

classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Naive Bayes Algo Accuracy:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

Naive Bayes Algo Accuracy: 90.63
Most Informative Features
                      Go = True                1 : 0      =     10.3 : 1.0
              Scriptures = True                1 : 0      =      9.5 : 1.0
                   anger = True                1 : 0      =      3.2 : 1.0
                   horse = True                1 : 0      =      2.5 : 1.0
                    wait = True                1 : 0      =      2.1 : 1.0
                rankings = True                1 : 0      =      1.9 : 1.0
                  report = True                1 : 0      =      1.4 : 1.0
                 abusive = True                0 : 1      =      1.3 : 1.0
                  poorly = True                0 : 1      =      1.2 : 1.0
                   steps = True                1 : 0      =      1.1 : 1.0
                accident = True                1 : 0      =      1.1 : 1.0
                    Well = True                1 : 0      =      1.1 : 1.0
                      Go = False         

In [54]:
a= pd.DataFrame(training_set)


In [56]:
a.nonzero()

AttributeError: 'DataFrame' object has no attribute 'nonzero'

In [20]:
#put all classifiers from above into 1, the new classifier that we will build will be based on the best (most vote for a classifier)

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [21]:
voted_classifier = VoteClassifier(classifier)
print("Voted Classifier Accuracy Percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)
print("Classification: ", voted_classifier.classify(testing_set[0][0]),
                                                    "Confidence % : ", voted_classifier.confidence(testing_set[0][0])*100)

Voted Classifier Accuracy Percent: 90.63
Classification:  0 Confidence % :  100.0


In [37]:
def isToxic(text):
    feats = find_features(text)
    
    return voted_classifier.classify(feats)

In [38]:
print(isToxic("Explanation"))

0


In [43]:
voted_classifier
